In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from selenium.common.exceptions import TimeoutException

# Assuming driver, username, password, and url are already defined
username = 'rene.scholten@rabobank.nl'
password = 'breeders'
url = 'https://fd.nl/login'
driver = webdriver.Edge()
# Open the login page
driver.get(url)

# Wait for the username field to be present
username_field = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.NAME, 'username'))
)
# Find and fill the username field
username_field.send_keys(username)

# Click the "next" button
next_button = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.XPATH, '//button[text()="Verder"]'))
)
next_button.click()

# Wait for the "log in met wachtwoord" button to be present and click it
login_with_password_button = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.XPATH, '//button[text()="Log in met wachtwoord"]'))
)
login_with_password_button.click()

# Wait for the password field to be present
password_field = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.NAME, 'password'))
)
# Find and fill the password field
password_field.send_keys(password)

# Submit the login form
password_field.send_keys(Keys.RETURN)

# Wait for the login to complete
time.sleep(10)

# Navigate to the search page
search_url = 'https://fd.nl/search?q=energie'
driver.get(search_url)

# Loop to handle multiple cookie consent windows
while True:
    try:
        # Wait for the cookie consent button to be present
        cookie_consent_button = WebDriverWait(driver, 2).until(
            EC.element_to_be_clickable((By.XPATH, '//button[text()="Alles accepteren"]'))
        )
        # Use the Enter key to click the button
        cookie_consent_button.send_keys(Keys.ENTER)
        time.sleep(2)  # Wait a bit for the consent to be processed
    except TimeoutException:
        break  # Exit the loop if no more cookie consent buttons are found

# Wait for the search results to load
time.sleep(10)
    
# Scrape the articles
articles = driver.find_elements(By.CLASS_NAME, 'search-result')

data = []
for article in articles:
    try:
        date = article.find_element(By.CLASS_NAME, 'date').text
        title = article.find_element(By.CLASS_NAME, 'title').text
        author = article.find_element(By.CLASS_NAME, 'author').text
        subject = article.find_element(By.CLASS_NAME, 'subject').text
        article_text = article.find_element(By.CLASS_NAME, 'article-text').text
        data.append({
            'date': date,
            'title': title,
            'author': author,
            'subject': subject,
            'article_text': article_text
        })
    except Exception as e:
        print(f"Error scraping article: {e}")

# Create a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

# Close the driver
driver.quit()

Empty DataFrame
Columns: []
Index: []
